# Modeling and Analysis of Total Fitness Factor Score for CSUF College Students

## Group Members
- Paul Anthony Bagabaldo
- Joksan Hernandez
- Huyen Nguyen

## Acknowledgements
Dr. Bill Beam (Department of Kinesiology), Dr. Archana McEligot (Department of Public Health), and Dr. Sinjini Mitra (Department of Information Systems and Decision Sciences) at California State University, Fullerton, for providing the data used in this project.

## Table of Contents 
<a id="TOC"></a>
1. [Introduction](#Introduction)
2. [Data Description](#Data-Description)
3. [Data Preprocessing](#Data-Preprocessing)
4. [Exploratory Data Analysis](#Exploratory-Data-Analysis)
5. [Modeling and Analysis](#Modeling-and-Analysis)
6. [Results](#Results)
7. [Conclusion](#Conclusion)
8. [References](#References)

## Introduction
<a id="Introduction"></a>

Physical Fitness plays a large role in the overall health and well being of college students. These factors influencing fitness can help in designing targeted interventions to promote healthier lifestyles. Researchers at California State University, Fullerton (CSUF), in collaboration with various departments, have collected data on health and fitness-related variables for a long-term study. 

The goal of this project is to approximate the **Total Fitness Factor Score (FFTotal)** using other variables in the dataset. In this project, we will be using **explatory data analysis**, **linear regression modeling**, and **performance evaluation** to better understand the data. 

## Data Description

The dataset consists of various health and fitness-related variables collected as part of fitness testing among students at California State University, Fullerton. Below is a summary of the key variables used in the analysis:

### Demographic Information
- **Idnum**: A random identification number assigned at the conclusion of the semester.
- **Date**: The date or semester during which the test was conducted.
- **Phone**: Last four digits of the self-reported phone number for matching data.
- **Sex**: Gender of the subject (Female = F, Male = M).
- **Age**: Self-reported age in years.
- **Ethnicity**: Self-reported ethnicity (categories include Caucasian, Hispanic/Latino, African American, Native American, Asian, Pacific Islander, or Other).

### Anthropometric Measurements
- **Height (Ht)**: Height measured using a stadiometer to the closest 0.25 inches.
- **Weight (Wt)**: Weight measured using an electronic scale to the closest 0.1 lb.
- **BIA % Fat**: Body fat percentage measured using bioelectrical impedance analysis (BIA).
- **Waist Girth**: Measured at the "minimal" natural waist in centimeters.
- **Skinfold Measurements (SF 1, SF 2, SF 3)**: Skinfold thickness (in millimeters) at various sites:
  - SF1: Chest (male) or triceps (female).
  - SF2: Abdomen (male) or suprailium (female).
  - SF3: Thigh (both genders).

### Fitness and Physical Performance
- **Forward Flexion (FF)**: Sit-and-reach test result, best of three trials, measured to the closest 0.5 inch.
- **Right Grip Max (RGM)** and **Left Grip Max (LGM)**: Maximal grip strength for each hand (in kilograms).
- **Vital Capacity (VC)**: Lung capacity measured using a Vitalometer (in liters).
- **Stages**: Number of stages completed on a cycle ergometer test (range: 2–4 stages).
- **Power and Heart Rate per Stage (PL 1-4, HR 1-4)**: Power (in watts) and heart rate (in bpm) for each stage of the cycle ergometer test.
- **Rate of Perceived Exertion (RPE 1-4)**: Self-reported exertion at the end of each stage.

### Cardiovascular and Environmental Data
- **Resting HR**: Resting heart rate (in bpm), self-reported or measured.
- **Systolic BP (SBP)** and **Diastolic BP (DBP)**: Brachial systolic and diastolic blood pressure (in mmHg), measured in a seated position.
- **Ambient Temperature (TA)**: Temperature of the testing environment in degrees Celsius.
- **Barometric Pressure (PB)**: Measured barometric pressure (in mmHg).

### Risk Factors
- **RF 2**: Gender-based risk factor (Female = 1, Male = 5).
- **RF 3**: Stress score based on 11 questions (range: 11–55).
- **RF 4**: Family history of cardiovascular disease (CVD) categorized into severity levels.
- **RF 5**: Smoking history, self-reported (categories based on number of cigarettes/day).

### Outcome Variable
- **Total Fitness Factor Score (FFTotal)**: The primary outcome variable, representing an overall fitness metric calculated using a proprietary formula.


[Back to Top](#TOC)

## Data Preprocessing

In [ ]:
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()

df = pd.read_csv('totalFitnessFactor.csv')

df = df.dropna(axis=1, how='all')

: 

The initial dataset was loaded from 'totalFitnessFactor.csv'. Empty columns were identified and removed. This step removed columns containing only NA values, reducing noise and improving data quality for analysis.

In [ ]:
# Calculate the missing statistics
missing_stats = (
   df.isnull()
       .sum()
       .sort_values(ascending=False)
       .to_frame('Missing Count')
       .join(
           (df.isnull().mean() * 100)
           .round(2)
           .to_frame('Missing %')
       )
       [lambda x: x['Missing Count'] > 0]
)

# Display the missing statistics
(
missing_stats
    .style
    .format({
        'Missing Count': '{:,.0f}',
        'Missing %': '{:.2f}%'
    })
    .background_gradient(cmap='Blues')
)

: 

After reviewing the dataset using Python, we found that the columns Waist and BIA % Fat had significant amounts of missing data, with 88.08% and 74.57% of their values missing, respectively. Given the high percentage of missing values, we decided to exclude these columns from the analysis.

In contrast, the columns SF 1, SF 2, and SF 3 had about 25.43% of their values missing. While this is not ideal, these columns still contain enough data to potentially provide valuable insights, so they were retained for further exploration.

Lastly, there were several other columns, such as PL 3 (6.05%), HR 3 (6.05%), and RPE 3 (6.26%), that had only a small proportion of missing data. Given that these gaps were minimal, we concluded that keeping these columns would not significantly impact the results.

In [84]:
# Calculate the missing percentage for each column
missing_percentage = df.isnull().mean() * 100

# Clean the data by removing columns with more than 30% missing values
df_cleaned = df.loc[:, missing_percentage <= 30]
df_cleaned.columns

#Display the retained columns
retained_cols = pd.DataFrame({
   'Column Name': df_cleaned.columns.tolist(),
   'Data Type': df_cleaned.dtypes.values,
   'Non-null Count': df_cleaned.count().values
}).set_index('Column Name')

(
retained_cols
   .style
   .set_table_styles([
       {'selector': 'thead',
        'props': [('background-color', '#2c3e50'), 
                 ('color', 'white'),
                 ('font-weight', 'bold')]},
   ])
   .format({'Non-null Count': '{:,d}'})
)

,Data Type,Non-null Count
Column Name,,
Idnum,object,"6,276"
Date,object,"6,276"
Sex,object,"6,276"
Age,int64,"6,276"
Ht,float64,"6,276"
Wt,float64,"6,276"
RF 2,int64,"6,276"
RF 3,int64,"6,276"
RF 4,int64,"6,276"


visualize 5 variables with FFTotal
using 'df_cleaned' we will visualize 5 variables along with FFTotal in order to see if there is a correlation between them to use for later calculations to predict FFTotal. Originally done in Google Colab but work copy andd pasted here.

1. compare variable 'Sex' vs 'FFTotal' to see the distributation of FFTotal across both genders. Used both a boxplot and histogram.

Boxplot:

Female subjects tend to have a lower FFTotal score than males.

Males have outliers within their distribution extending past the lower fence.

Histogram:

Unimodal distribution indicates FFTotal for both sexes mainly range between 25-45.

Symmetric distrubition indicates data has a normal distribution where most data points cluster around the middle and taper off towards both ends. 

**Analysis:**

Since there is a clear distribution indicated in both the boxplot and histogram charts, we can conclude that Sex might be an important factor in calculating FFTotal.
We need to take into account gender when calculating FFTotal scores since we can see that females on average generally have a lower score than males.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(data=df_cleaned, x='Sex', y='FFTotal')
plt.title('Distribution of FFTotal by Sex')
plt.show()

In [ ]:
sns.histplot(data=df_cleaned, x='FFTotal', hue='Sex', kde=True)
plt.title('Distribution of FFTotal by Sex')
plt.show()

2. compare variable 'Age' vs 'FFTotal' to see if there is a correlation between the two variables.

Boxplot:

Boxplot distribution between ages aren't very different, indicates that FFTotal can vary and is not dependent on Age. The sample age range is also very small (18-25) so that's another reason why we probably don't see much impact on FFTotal regarding age.

Scatterplot:

Age ranges from 18-25, very limited range so we don't know if Age would be a useful variable to use when calculating FFTotal since we don't have older test sujects to gauge the difference in young vs old age.

There seems to be no apparent relationship between Age and FFTotal as there are points at every range of FFTotal between all ages.

**Analysis:**

We would reccomend against using this variable as part of calculating FFTotal since there seems to be no correlation between Age and FFTotal and the sample age range is also limited so we can't truly see the impact young vs old age would have on the fitness score

In [ ]:
sns.boxplot(data=df_cleaned, x='Age', y='FFTotal')
plt.title('boxplot: Age vs. FFTotal')
plt.show()

In [ ]:
sns.scatterplot(data=df_cleaned, x='Age', y='FFTotal')
plt.title('scatterplot: Age vs. FFTotal')
plt.show()

3. compare variable 'HR rest' vs 'FFTotal' to see if a lower resting heart rate would indicate a higher total fitness score. 

In [ ]:
sns.scatterplot(data=df_cleaned, x='HR rest', y='FFTotal')
plt.show()

4. compare variable 'Weight' vs 'FFTotal' to see if there is a correlation between the two.

Scatterplot:

There are a lot of varying datapoints for weights less than 200lbs, showing that fitness scores can still vary if weight isn't too high.

At weights over 250, we can see that there are little to no datapoints with FFTotal scores of more than 40.

**Analysis:** 

The scatterplot shows a slight negative correlation between Weight and FFTotal meaning that as weight increases, we can generally expect the FFTotal to decrease. Since data points are still fairly scattered between varying weights, we can assume there is little correlation between the two variables and should keep in mind only use it to some extent in our equation to calculate FFTotal. We can increase the importance of Weight in calculating the FFTotal as it increasing and goes above a certain threshold as we can see that higher weights can generally indicate lower FFTotal scores.

In [ ]:
sns.scatterplot(data=df_cleaned, x='Wt', y='FFTotal')
plt.title('Weight vs. FFTotal')
plt.show()

5. compare variable 'Stages' vs 'FFTotal' where stages represents the number of stages completed on cycle ergometer test (range 2-4). A boxplot will be used to illustrate the distribution of FFTotal scores depending on the number stages a subject can complete.

Boxplot:

FFTotal scores increase as the stages increase. Subjects who are able to complete a higher number of stages will have an increased likelihood of having a higher FFTotal.

**Analysis:**

We can see that there is a clear correlation between 'Stages' and 'FFTotal' as the boxplot indicates a clear increase in fitness scores as the stages increase. With this knowledge, we can safely assume that 'Stages' will be an important variable in calculating FFTotal and can make sure to include it as part of our equation.

In [ ]:
sns.boxplot(data=df_cleaned, x='Stages', y='FFTotal')
plt.show()